In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import Session, sessionmaker

from algolink.exp.sqlalchemy.models import Base

class SQLAlchemyMetaRepository(object):
    """
    :class:`.MetadataRepository` implementation which stores metadata in SQL database via `sqlalchemy` library.

    :param db_uri: URI of SQL database to connect to
    """

    type = 'sqlalchemy'
    def __init__(self, db_uri: str):
        self.db_uri = db_uri
        self._engine = create_engine(db_uri)
        Base.metadata.create_all(self._engine)
        self._Session = sessionmaker(bind=self._engine)
        self._active_session = None

    @contextlib.contextmanager
    def _session(self) -> Session:
        if self._active_session is None:
            logger.debug('Creating session for %s', self.db_uri)
            self._active_session = self._Session()
            new_session = True
        else:
            new_session = False

        try:
            yield self._active_session

            if new_session:
                self._active_session.commit()
        except:  # noqa
            if new_session:
                self._active_session.rollback()
            raise
        finally:
            if new_session:
                self._active_session.close()
                self._active_session = None
                
    def _get_objects(self, object_type: Type[Attaching], add_filter=None) -> List:
        with self._session() as s:
            if add_filter is None:
                logger.debug('Getting %ss', object_type.__name__)
            else:
                logger.debug('Getting %ss with filter %s', object_type.__name__, add_filter)
            q = s.query(object_type)
            if add_filter:
                q = q.filter(add_filter)
            return [o.to_obj() for o in q.all()]

In [3]:
from algolink.ext.sqlalchemy.repository import SQLAlchemyMetaRepository
from typing import List, Optional, Type, TypeVar, Union
from algolink.ext.sqlalchemy.models import Attaching,TExperiment

class AlgoLinkUI(SQLAlchemyMetaRepository):
    def __init__(self,db_uri: str):
        super().__init__(db_uri)

    def _get_objects(self, object_type: Type[Attaching], add_filter=None) -> List:
        with self._session() as s:
            if add_filter is None:
                logger.debug('Getting %ss', object_type.__name__)
            else:
                logger.debug('Getting %ss with filter %s', object_type.__name__, add_filter)
            q = s.query(object_type)
            if add_filter:
                q = q.filter(add_filter)
            return [o.to_obj() for o in q.all()]
        
        
test = AlgoLinkUI("sqlite:///sql.db")

ValueError: Cant register <class '__main__.AlgoLinkUI'> as __main__.AlgoLinkUI. Subtype <class '__main__.AlgoLinkUI'> is already registered

In [13]:
from algolink.ext.sqlalchemy.repository import SQLAlchemyMetaRepository
from algolink.ext.sqlalchemy.models import Attaching,TExperiment,STask

test = SQLAlchemyMetaRepository("sqlite:///sql.db")



In [11]:
exp=test._get_objects(TExperiment,TExperiment.id)

In [14]:
from pprint import pprint
from pyjackson import serialize

pprint(serialize(exp))

[{'author': 'leepand',
  'creation_date': '2021-10-14 02:14:24.058297 ',
  'experiment_remark': 'experiment_remark',
  'experiment_sequence': 1,
  'id': 1,
  'name': 'test_track5',
  'task_id': 2},
 {'author': 'leepand',
  'creation_date': '2021-10-15 09:53:15.620394 ',
  'experiment_remark': 'experiment_remark',
  'experiment_sequence': 1,
  'id': 2,
  'name': 'test_track1',
  'task_id': 5}]


In [30]:
x = []
for i in exp:
    exp2=test._get_object_by_id(STask,i.task_id)
    x.append(exp2)
    #print(i.task_id)
    pprint(serialize(exp2))

{'author': 'leepand',
 'creation_date': '2021-10-14 02:14:24.039844 ',
 'datasets': {},
 'evaluation_sets': {},
 'id': 2,
 'metrics': {},
 'name': 'my_task_tracking',
 'project_id': 1}
{'author': 'leepand',
 'creation_date': '2021-10-15 09:28:05.955313 ',
 'datasets': {},
 'evaluation_sets': {},
 'id': 5,
 'metrics': {},
 'name': '机器学习任务',
 'project_id': 2}


In [31]:
x

[Task(id=2,name=my_task_tracking), Task(id=5,name=机器学习任务)]

In [29]:
test._get_object_by_id(STask,5)

Task(id=5,name=机器学习任务)

In [32]:
!pwd

/Users/leepand/Downloads/MLOps/AlgoLink/tests


In [62]:
!ls -a .ebonite/artifacts/model/0

.                 methods.json      requirements.json
..                model.pkl


In [63]:
import pandas as pd
import sqlite3

con = sqlite3.connect("sql.db")

# Load the data into a DataFrame
surveys_df = pd.read_sql_query("SELECT * from experiments", con)

# Select only data for 2002
#surveys2002 = surveys_df[surveys_df.year == 2002]

# Write the new DataFrame to a new SQLite table
#surveys2002.to_sql("surveys2002", con, if_exists="replace")

con.close()

In [64]:
surveys_df

,id,name,author,experiment_remark,experiment_sequence,del_flag,creation_date,task_id
0,1,test_track5,leepand,experiment_remark,1,0,2021-10-16 07:50:31.206202,1
1,2,test_track6,leepand,experiment_remark,1,0,2021-10-16 08:28:50.595045,2
2,3,test_track7,leepand,experiment_remark,2,0,2021-10-16 16:13:00.386830,2
3,4,test_track9,leepand,experiment_remark,2,0,2021-10-17 10:18:50.957025,1
4,5,test_track10,leepand,experiment_remark,3,0,2021-10-17 10:32:02.063523,1


In [65]:
from sqlalchemy import create_engine

disk_engine = create_engine('sqlite:///sql.db')
#price.to_sql('stock_price', disk_engine, if_exists='append')
df = pd.read_sql_query('SELECT * FROM experiments LIMIT 3',disk_engine)
df.head()

,id,name,author,experiment_remark,experiment_sequence,del_flag,creation_date,task_id
0,1,test_track5,leepand,experiment_remark,1,0,2021-10-16 07:50:31.206202,1
1,2,test_track6,leepand,experiment_remark,1,0,2021-10-16 08:28:50.595045,2
2,3,test_track7,leepand,experiment_remark,2,0,2021-10-16 16:13:00.386830,2
